In [13]:
import sqlite3


O Banco de Dados em formato sqlite possuí a tabela SO_QUESTION que contém as colunas: 

In [14]:
conn = sqlite3.connect('dados/tags_database.db')
cursor = conn.cursor()

In [20]:
cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\';')
print (cursor.fetchone())
print (cursor.fetchone())
print (cursor.fetchone())

('SO_QUESTION',)
('SO_TAG',)
('SO_QUESTION_TAG',)


In [21]:

cursor.execute('SELECT * FROM SO_QUESTION')
row = cursor.fetchone()
names = cursor.description
print (names)
print (row)
row = cursor.fetchone()
print (row)

(('ID', None, None, None, None, None, None), ('SO_ID', None, None, None, None, None, None), ('OWNER_ID', None, None, None, None, None, None), ('CREATION_DATE', None, None, None, None, None, None))
(1, '4', '8', '2008-07-31 21:42:52.667000')
(2, '6', '9', '2008-07-31 22:08:08.620000')
